# Load, Read and Analyze ERA5 Data

First, import some required packages

In [ ]:
%matplotlib widget
import os
import ee
import geemap
import requests
import cftime
import numpy as np
import pandas as pd
import netCDF4 as nc
import datetime as dt
import rasterio as rio
import cartopy.crs as ccrs
import cartopy.util as cutil
import matplotlib.pyplot as plt

from matplotlib import cm
from rasterio import plot
from rasterio import warp
from shapely import geometry
from datetime import timedelta
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import axes3d
from cartopy.io.img_tiles import Stamen
from ipyleaflet import basemaps, basemap_to_tiles

In [ ]:
def nc_read(filename, variable):
    '''
    Read variable data from a NetCDF source
    :param filename:(string) complete path and file name
    :param variable: (string) name of variable
    :return: numpy array containing data
    '''
    
    data = nc.Dataset(filename)
    var = np.squeeze(data[variable][:])
    return var

In [ ]:
def return_variable_data(file, var, t_units):
    '''
    Return the time, latitude, longitude and variable values for the netcdf4 file
    :param file: (string) the complete path and file name
    :param var: (string) the variable name as defined in the NetCDF4 file.
    :param t_units: (string) the units for the time
    :return time: numpy array containing time data
    :return lats: numpy array containing latitude data
    :return lons: numpy array containing longitude data
    :return var: numpy array containing data for the variable of interest
    '''
    time = nc_read(file, 'time')
    lats = nc_read(file, 'latitude')
    lons = nc_read(file , 'longitude')
    var = nc_read(file, var)

    time.units = t_units
    time = nc.num2date(time, time.units)
    
    return time, lats, lons, var

In [ ]:
def spatial_extent_reduction(lat, lon, var):
    '''
    Reduces the variable data to the Pine Island Glacier Polynya region.
    :param lat: numpy array containing original latitude data
    :param lon: numpy array containing original longitude data
    :param lat: numpy array containing original variable data
    :return: The same input variable for the reduced region
    '''
    lat = lat[3:6]
    lon = lon[:-2]
    var = var[:,:,3:6,:-2]
    return lat, lon, var

In [ ]:
def load_data(var, spatial_reduction=0):
    '''
    File that loads the desired ERA5 data, and reduces the spatial extent if required.
    :param var: (string) variable name of interest
    :param spatial_reduction: (int) Can take values 0 or 1 where there is no spatial reduction for 0 and spatial reduction for 1
    :return: Processed ERA5 data for the specified variable of interest
    '''
    file = '/home/jovyan/Polynyas/data/ERA5/PineIslandPolynya_ERA5_data.nc'
    t_units = 'hours since 1900-01-01'
    time, lat, lon, var = return_variable_data(file, var, t_units)
    if spatial_reduction ==1:
        lat, lon, var = spatial_extent_reduction(lat, lon, var)
        return time, lat, lon, var
    else:
        return time, lat, lon, var
    

In [ ]:
def quick_visualisation(vardata, lon, lat, varname, date, filled_contours=0):
    '''
    Creates a contour plot for the variable data of interest.
    :param vardata: numpy array containing data for the variable of interest.
    :param lon: 1D numpy array containing longitude data 
    :param lat: 1D numpy array containing latitude data 
    :param varname: (string) the variable name for plotting
    :param date: (string) the time period spanned by the variable data
    :param filled_contours: set to 0 if no filled contours, set to 1 for filled contours
    '''
    
    lons, lats = np.meshgrid(lon, lat)
    plt.style.use('seaborn-bright')
    tiler = Stamen('terrain-background')
    mercator = tiler.crs

    fig = plt.figure(figsize=[14,14])
    ax = fig.add_subplot(1, 1, 1, projection=mercator)
    
    ax.set_extent([ -100, -102, -75.75, -74.25], ccrs.PlateCarree())

    if filled_contours==0:
        print("no filled contours")
    elif filled_contours==1:
        filled_c = ax.contourf(lons, lats, vardata,transform=ccrs.PlateCarree(),cmap='nipy_spectral')
        fig.colorbar(filled_c, orientation='horizontal')
    
    line_c = ax.contour(lons, lats, vardata, colors=['black'],transform=ccrs.PlateCarree())
    ax.clabel(line_c, colors=['black'], manual=False, inline=True, fmt=' {:.0f} '.format)
    geom = geometry.box(minx=-101.8,maxx=-100.5,miny=-75.25,maxy=-74.8)
    ax.add_geometries([geom], crs=ccrs.PlateCarree(), facecolor='r', edgecolor='black', alpha=0.3)
    ax.add_image(tiler, 6)
    ax.coastlines()
    ax.set_title('Quick Visualisation of {}'.format(varname)+'\n for {}'.format(date), fontsize=20, fontweight='bold', pad=20)


In [ ]:
def quick_visualisation_winds(u, v, lon, lat, varname, date):
    '''
    Creates a vector plot for the 10m wind.
    :param u: numpy array containing zonal wind component data
    :param v: numpy array containing meridional wind component data
    :param lon: 1D numpy array containing longitude data 
    :param lat: 1D numpy array containing latitude data 
    :param varname: (string) the variable name for plotting
    :param date: (string) the time period spanned by the variable data
    '''
    lons, lats = np.meshgrid(lon, lat)
    plt.style.use('seaborn-bright')
    tiler = Stamen('terrain-background')
    mercator = tiler.crs

    fig = plt.figure(figsize=[14,14])
    ax = fig.add_subplot(1, 1, 1, projection=mercator)
    
    ax.set_extent([ -100, -102, -75.75, -74.25], ccrs.PlateCarree())
    ax.quiver(lon,lat,u, v,units='xy', headwidth=3, transform=ccrs.PlateCarree())
    geom = geometry.box(minx=-101.8,maxx=-100.5,miny=-75.25,maxy=-74.8)
    ax.add_geometries([geom], crs=ccrs.PlateCarree(), facecolor='r', edgecolor='black', alpha=0.3)
    ax.add_image(tiler, 6)
    ax.coastlines()
    ax.set_title('Quick Visualisation of {}'.format(varname)+'\n for {}'.format(date), fontsize=20, fontweight='bold', pad=20)
    

In [ ]:
def quick_visualisation_ts(vardata, varname, time):
    '''
    Creates a timeseries plot for the variable of interest.
    :param vardata: numpy array containing the variable time series data
    :param varname: (string) the variable name for plotting
    :param time: (string) the time period spanned by the variable time series data
    '''
    plt.style.use('seaborn-bright')
    plt.figure(figsize=[20,10])
    time = pd.DataFrame(time)
    vardata = pd.DataFrame(vardata)
    vardata_weekly = vardata.rolling(7).mean()
    vardata_monthly = vardata.rolling(30).mean()
    
    plt.plot(vardata, color='k', linewidth=0.5, label='Daily MSLP')
    plt.plot(vardata_weekly, color='b', linewidth=1.5, label='Weekly MSLP')
    plt.plot(vardata_monthly, color='r', linewidth=1.5, label='Monthly MSLP')
    
    plt.title('Mean Sea Level Pressure over the Pine Island Polynya' , fontsize=22, fontweight='bold')
    plt.xlabel('Time', fontsize=20)
    plt.ylabel('Pressure (Pa)', fontsize=20)
    #plt.xticks(ticks=time, rotation=60, fontsize=22)
    plt.legend(fontsize=18)


In [ ]:
def return_date_index(date, time):
    '''
    Returns the index for the date of interest
    :param date: date of interest
    :param time: time data
    '''
    i = np.argwhere(time == date)
    i = np.ravel(i)
    return i[0]

In [ ]:
def temporal_mean(period, vardata, time, day=0):
    '''
    Takes the temporal mean of the variable data for the given period. For user friendliness, we can pick either 'weekly', 'monthly', 'yearly', 'overall' or 'seasonal' averages. 
    The 'seasonal' average will prompt the user to also include the 'season' of interest. For now, the function only calculates the temporal mean for the overall time period.
    :param period: (string) time period of interest
    :param vardata: numpy array containing variable data
    :param time: numpy array containing time data
    :param day: set to 0 if we are not extracting a single day and to 1 if we are.
    :return: temporal mean of variable data for time period chosen
    '''

    if period == 'overall':
        vardata_mn = np.mean(vardata, axis = 0)

        
    return vardata_mn[0,:,:]

In [ ]:
def spatial_mean(vardata, lat, lon):
    '''
    Calculates the spatial mean of the data for the Polynya bounding box region.
    '''
    lat, lon, vardata_ = spatial_extent_reduction(lat, lon, vardata)
    vardata_mn = np.mean(vardata_[:,0,:,:], axis = 2)
    vardata_mn = np.mean(vardata_mn[:,:], axis=1)
    
    return vardata_mn

Options for variables, and corresponding units and variable names
- Mean sea level pressure (Pa): 'msl'
- Mean surface latent heat flux (W m**(-2)): 'mslhf'
- Mean surface sensible heat flux (W m**(-2)): 'msshf'
- Surface sensible heat flux (J m**(-2)): 'sshf'
- Sea surface temperature (K): 'sst'
- Zonal wind 10m above surface (m/s): 'u10'
- Meridional wind 10m above surface (m/s): 'v10'

Note: If you are unfamiliar with heat fluxes, a negative heat flux means heat is flowing from the surface to the atmosphere, a positive heat flux means heat is flowing from the atmosphere to the surface.

## Let's look at some variables of interest.
### First let's set the time frame of interest:

In [ ]:
# Specify the period of interest to a specific day, weekly mean, monthly mean or yearly mean
period = "day"
if period == "day":
    set_date = cftime.DatetimeGregorian(2019, 4, 15, 15, 0, 0, 0, has_year_zero=False)

### Let's investigate the mechanical driving of the Polynya
#### First we loop at the mean sea level pressure

In [ ]:
# Load the data for the variable of interest
var = 'msl'
varname = 'Mean Sea Level Pressure (Pa)'
time, lats, lons, vardata = load_data(var)

#The following code isn't working and can't figure it out so will do this the non-modular way for now
# Do some data extraction based on the time frame of interest
#date_i, time_, vardata_ = temporal_mean(period, vardata, time, day= set_date)

#extract index for date of interest set_date
date_i = return_date_index(set_date, time)

#plot the msl data for this day
quick_visualisation(vardata[date_i,0,:,:], lons, lats, varname, time[date_i], filled_contours = 0)


#### Which can be further interpreted by the wind vectors

In [ ]:
#set wind vector variable names
var_u = 'u10'
var_v = 'v10'
varname = '10m Winds'
#extract wind data
time, lats, lons, vardata_u = load_data(var_u)
time, lats, lons, vardata_v = load_data(var_v)
#visualize wind data
quick_visualisation_winds(vardata_u[date_i,0,:,:], vardata_v[date_i,0,:,:], lons, lats, varname, time[date_i])

### Now we can look at some energy fluxes

In [ ]:
#define the heat flux variable names of interest
lh = 'mslhf'
sh = 'msshf'

lhname = 'Mean Surface Latent Heat Flux (W m**(-2))'
shname = 'Mean Surface Sensible Heat Flux (W m**(-2))'

# Load the data for the variables of interest
time, lats, lons, lhdata = load_data(lh)
time, lats, lons, shdata = load_data(sh)

#return index for date of interest, set_date
date_i = return_date_index(set_date, time)

#calculate the net heat flux from the sensible and latent heat fluxes
nethf = (lhdata[date_i,0,:,:] + shdata[date_i,0,:,:])
nethfname = 'Net Mean Surface Heat Flux (W m**(-2))'

#visualize the results
quick_visualisation(lhdata[date_i,0,:,:], lons, lats, lhname, time[date_i], filled_contours = 1)
quick_visualisation(shdata[date_i,0,:,:], lons, lats, shname, time[date_i], filled_contours = 1)
quick_visualisation(nethf, lons, lats, nethfname, time[date_i], filled_contours = 1)

### So it seems like the surface heat flux variables do not necessarily capture the Polynya
#### Let's investigate whether any features are captured on average

In [ ]:
#define the heat flux variable names of interest
lh = 'mslhf'
sh = 'msshf'

lhname = 'Mean Surface Latent Heat Flux (W m**(-2))'
shname = 'Mean Surface Sensible Heat Flux (W m**(-2))'

# Load the data for the variables of interest
time, lats, lons, lhdata = load_data(lh)
time, lats, lons, shdata = load_data(sh)

# calculate the overall temporal mean for the data
lhdata_mn = temporal_mean('overall', lhdata, time, day=0)
shdata_mn = temporal_mean('overall', shdata, time, day=0)

#calculate the net heat flux from the sensible and latent heat fluxes
nethf_mn = (lhdata_mn[:,:] + shdata_mn[:,:])
nethfname = 'Net Mean Surface Heat Flux (W m**(-2))'

#visualize the results
quick_visualisation(lhdata_mn, lons, lats, lhname, 'Mean for January 2018 - March 2022', filled_contours = 1)
quick_visualisation(shdata_mn, lons, lats, shname, 'Mean for January 2018 - March 2022', filled_contours = 1)
quick_visualisation(nethf_mn, lons, lats, nethfname, 'Mean for January 2018 - March 2022', filled_contours = 1)

#### So it's become obvious that while the ERA5 data may help us understand some of the physical mechanisms that contribute to the formation of the Polynya, it cannot resolve the heat flux dynamics over the region. An extension would be to look at the time series of heat flux in the Polynya region and see if the behaviour of the time series mimicks the observed behaviour of the Polynya.

### Given that the ERA5 sea level pressure and winds are more interpretable, we can look at the time series for the Polynya region and assess whether the behaviour mimicks what is observed. 

In [ ]:
# Load the data for the variable of interest
var = 'msl'
varname = 'Mean Sea Level Pressure (Pa)'
time, lats, lons, vardata = load_data(var)
vardata_mn = spatial_mean(vardata, lats, lons)

quick_visualisation_ts(vardata_mn, varname, time)


## Now let's move on to the three dimensional plots with ICESat-2 Tracks

Here we define the dataCollector class based off of Philipp Arndt's code seen in visualisation tutorial. We define it within the code since there were issues importing the class from an outside python script.

In [ ]:
class dataCollector:
    def __init__(self, beam=None, oaurl=None, track=None, date=None, latlims=None, lonlims=None, verbose=False):
        if (beam is None) or ((oaurl is None) and (None in [track, date, latlims, lonlims])):
            raise Exception('''Please specify a beam and 
            - either: an OpenAltimetry API url, 
            - or: a track, date, latitude limits and longitude limits.''')
        else:
            if oaurl is not None:
                url = oaurl
                tofind = '&beamName='
                ids = url.find(tofind)
                while ids>-1:
                    url = url.replace(url[ids:ids+len(tofind)+4],'')
                    ids = url.find(tofind)
                iprod = url.find('/atl')
                url = url.replace(url[iprod:iprod+6],'/atlXX')
                url += tofind + beam + '&client=jupyter'

                idate = url.find('date=') + len('date=')
                date = url[idate:idate+10]
                itrack = url.find('trackId=') + len('trackId=')
                trackend = url[itrack:].find('&')
                track = int(url[itrack:itrack+trackend])
                bb = []
                for s in ['minx=', 'maxx=', 'miny=', 'maxy=']:
                    ids = url.find(s) + len(s)
                    ide = url[ids:].find('&')
                    bb.append(float(url[ids:ids+ide]))
                lonlims = bb[:2]
                latlims = bb[2:]
            elif None not in [track, date, latlims, lonlims]:
                url = 'https://openaltimetry.org/data/api/icesat2/atlXX?'
                url += 'date={date}&minx={minx}&miny={miny}&maxx={maxx}&maxy={maxy}&trackId={track}&beamName={beam}'.format(
                        date=date,minx=lonlims[0],miny=latlims[0],maxx=lonlims[1],maxy=latlims[1],track=track,beam=beam)
                url += '&outputFormat=json&client=jupyter'
            
            self.url = url
            self.date = date
            self.track = track
            self.beam = beam
            self.latlims = latlims
            self.lonlims = lonlims
            if verbose:
                print('OpenAltimetry API URL:', self.url)
                print('Date:', self.date)
                print('Track:', self.track)
                print('Beam:', self.beam)
                print('Latitude limits:', self.latlims)
                print('Longitude limits:', self.lonlims)
                
    def requestData(self, verbose=False): 
        if verbose:
            print('---> requesting ATL03 data...',end='')
        product = 'atl03'
        request_url = self.url.replace('atlXX',product)
        data = requests.get(request_url).json()
        lat, lon, h, confs = [], [], [], []
        for beam in data:
            for confidence in beam['series']:
                for p in confidence['data']:
                    confs.append(confidence['name'])
                    lat.append(p[0])
                    lon.append(p[1])
                    h.append(p[2])
        self.atl03 = pd.DataFrame(list(zip(lat,lon,h,confs)), columns = ['lat','lon','h','conf'])
        if verbose:
            print(' Done.')
            
            print('---> requesting ATL06 data...',end='')
        product = 'atl06'
        request_url = self.url.replace('atlXX',product)
        data = requests.get(request_url).json()
        self.atl06 = pd.DataFrame(data['series'][0]['lat_lon_elev'], columns = ['lat','lon','h'])
        if verbose:
            print(' Done.')
            
            print('---> requesting ATL07 data...',end='')
        product = 'atl07'
        request_url = self.url.replace('atlXX',product)
        data = requests.get(request_url).json()
        self.atl07 = pd.DataFrame(data['series'][0]['lat_lon_elev'], columns = ['lat','lon','h'])
        if verbose:
            print(' Done.')
            
            print('---> requesting ATL08 data...',end='')
        product = 'atl08'
        request_url = self.url.replace('atlXX',product)
        data = requests.get(request_url).json()
        self.atl08 = pd.DataFrame(data['series'][0]['lat_lon_elev_canopy'], columns = ['lat','lon','h','canopy'])
        if verbose:
            print(' Done.')

In [ ]:
try:
    ee.Initialize()
except: 
    ee.Authenticate()
    ee.Initialize()

In [ ]:
def data_from_oa_url(url,gtx):
    mydata = dataCollector(oaurl=url,beam=gtx)
    mydata.requestData()
    return (mydata)

We define a function to extract the satellite data we are interested in 

In [ ]:
#define url and gtx we desire
PIG_20201007_url = 'http://openaltimetry.org/data/api/icesat2/atl08?date=2020-10-07&minx=-101.06993207685464&miny=-75.06319452393107&maxx=-100.2946192252&maxy=-74.74165227279721&trackId=210&beamName=gt3r&beamName=gt3l&beamName=gt2r&beamName=gt2l&beamName=gt1r&beamName=gt1l&outputFormat=json'
PIG_20201007_gtx = 'gt2r'
#download the data 
PIG_20201007_data = data_from_oa_url(PIG_20201007_url, PIG_20201007_gtx)
#extract the data for ATL06 including, longitude, latitude and height
atl06_data = PIG_20201007_data.atl06
atl06_lat = atl06_data['lat']
atl06_lon = atl06_data['lon']
atl06_h = atl06_data['h']

#define a meshgrid for the region
new_lons, new_lats = np.meshgrid(atl06_lon, atl06_lat)

Given that we want to visualize this data together with the ERA5 data, we also load the ERA5 data for mean sea level pressure for the same date and interpolate it to the same grid size to be able to more easily code a visualization.

In [ ]:
var = 'msl'
varname = 'Mean Sea Level Pressure (Pa)'
time, lat, lon, vardata = load_data(var)
old_lon, old_lat = np.meshgrid(lon, lat)


date_i = return_date_index(set_date, time)

vardata_old = vardata[date_i,0,:,:]
time_ = time[date_i]

#interp
new_grid=griddata((old_lon.flatten(),old_lat.flatten()),vardata_old.flatten() , (new_lons,new_lats),method='cubic')


#### The rest is a work in progress

In [ ]:
fig    = plt.figure()
ax     = fig.gca(projection='3d')

#x      = np.linspace(0, 1, 100)
#X, Y   = np.meshgrid(x, x)
#levels = np.linspace(-0.1, 0.4, 100)  #(z_min,z_max,number of contour),

#a=0
#b=1
#c=2
#Z1 = a+.1*np.sin(2*X)*np.sin(4*Y)
#Z2 = b+.1*np.sin(3*X)*np.sin(4*Y)
#Z3 = c+.1*np.sin(4*X)*np.sin(5*Y)

#plt.contourf(old_lon, old_lat, vardata_old, levels =100, zdir='z',cmap=plt.get_cmap('nipy_spectral'))
#line_c = plt.contour(old_lon, old_lat, vardata_old, colors=['black'])
#ax.clabel(
#        line_c,  # Typically best results when labelling line contours.
#        colors=['black'],
#        manual=False,  # Automatic placement vs manual placement.
#        inline=True,  # Cut the line where the label will be placed.
#        fmt=' {:.0f} '.format,  # Labes as integers, with some extra space.
#    )
#plt.p

plt.contourf(new_lons, new_lats, new_grid, levels =100, cmap=plt.get_cmap('nipy_spectral'))
#line_c = plt.contour(new_lons, new_lats, new_grid, colors=['black'])
#plt.clabel(
#        line_c,  # Typically best results when labelling line contours.
#        colors=['black'],
 #       manual=False,  # Automatic placement vs manual placement.
  #      inline=True,  # Cut the line where the label will be placed.
   #     fmt=' {:.0f} '.format,  # Labes as integers, with some extra space.
    #)
#plt.plot(atl06_lon, atl06_lat, atl06_h, 'red')
#plt.contourf(X, Y,Z3, levels=c+levels,cmap=plt.get_cmap('rainbow'))
#ax.set_xlim3d(0, 1)
#ax.set_ylim3d(0, 1)
#ax.set_zlim3d(0, 2)

plt.show()

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot3D(atl06_lon, atl06_lat, atl06_h, 'red')
